In [68]:
import panel as pn
import json
import os
import pandas as pd
from collections import defaultdict


In [69]:
ch_dir = '/home/infosebi/Documents/Programmieren/MastersProject/test_3'

In [71]:
os.chdir(ch_dir)
sequence_df = pd.read_csv('sequence_stats.csv', index_col=0)
gene_df = pd.read_csv('genome_to_protein.csv', index_col=0)
cluster_df = pd.read_csv('cluster_to_gene.csv', index_col=0, converters = {'genes': pd.eval})
recluster_df = pd.read_csv('reclusters.txt', sep='\t', index_col=0, names=['Cluster', 'Protein'])

genome_list = []
genome_select = []

for index, row in sequence_df.iterrows():
    genome_list.append({index: row['protein count']})
    genome_select.append(index)
    
atlas_view = pn.pane.HTML("""
<iframe src="https://geneatlas.web.app/" id="myFrame" height="900" width="850" allow_embedding=True>
</iframe>
""",
styles={'background-color': '#F6F6F6',
'border': '2px solid black',
'border-radius': '5px',
'padding': '10px'}, height=900)
html_helper = pn.pane.HTML(" ")

def send_atlas(event):
    reference_genome = select_ref.value
    cluster_dict_list = []
    for index, row in gene_df.loc[(gene_df['genome'] == reference_genome) & (gene_df['type'] == 'protein')].iterrows():
        cluster_dict  = defaultdict(list)
        cluster_index = recluster_df.loc[recluster_df['Protein']==index].index[0]
        gene_info = cluster_df.loc[cluster_index]['genes']
        for gene in gene_info:
            cluster_dict[gene_df.loc[gene]['genome']].append(int(gene_df.loc[gene]['protein number']))
        cluster_dict_list.append(dict(cluster_dict))
    atlas_json = {"genomes": genome_list, "gensets": cluster_dict_list}
    atlas_json = json.dumps(atlas_json)

    html_helper.object = f"""<script type="text/javascript">
    var panel_row_elements = document.getElementsByClassName('bk-Row');
    console.log(panel_row_elements);
    var myFrame = panel_row_elements[0].shadowRoot.childNodes[9].shadowRoot.childNodes[8].childNodes[0];
    console.log(myFrame);
    myFrame.contentWindow.postMessage(`{atlas_json}`, '*');
    </script>"""
    html_helper.param.trigger('object')

select_ref = pn.widgets.Select(name='Select', options=genome_select)
atlas_button = pn.widgets.Button(name='Draw Geneatlas', width=150, button_type='primary')
atlas_button.on_click(send_atlas)

pn.Row(atlas_view, select_ref ,atlas_button, html_helper).show()

Launching server at http://localhost:38723
